In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import zipfile
import io
import os

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
file_path = '/content/drive/MyDrive/dataset/dataset_1.zip'
zip_read = zipfile.ZipFile(file_path, 'r')
zip_read.extractall('/content/dataset')
zip_read.close()

In [9]:
os.listdir('/content/dataset/dataset_1')

['test', 'train']

In [10]:
TRAINING_DIR = '/content/dataset/dataset_1/train'
VALIDATION_DIR = '/content/dataset/dataset_1/test'

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    vertical_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, class_mode='categorical',target_size=(150,150))

validation_datagen = ImageDataGenerator(
    rescale=1.0/255
)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, class_mode='categorical',target_size=(150,150))

Found 550 images belonging to 5 classes.
Found 122 images belonging to 5 classes.


In [11]:
pre_trained_model = MobileNetV2(weights="imagenet", include_top=False,
                                input_tensor=Input(shape=(150,150,3)))

for layer in pre_trained_model.layers:
  layer.trainable = False

last_output = pre_trained_model.output

x = tf.keras.layers.Flatten(name="Flatten")(last_output)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(128, activation="relu")(x)
x = tf.keras.layers.Dense(5, activation="softmax")(x)

model = tf.keras.models.Model(pre_trained_model.input, x)

int_lr = 1e-4
num_epochs = 30

optimizer = tf.optimizers.Adam(learning_rate=int_lr)
model.compile(
    optimizer=optimizer,
    loss="categorical_crossentropy",
    metrics=['accuracy']
)

H = model.fit(train_generator,
              epochs=num_epochs,
              validation_data=validation_generator)

9406464/9406464 [==============================] - 0s 0us/step
Epoch 1/30
18/18 [==============================] - 29s 1s/step - loss: 1.3921 - accuracy: 0.5964 - val_loss: 0.6705 - val_accuracy: 0.8115
Epoch 2/30
18/18 [==============================] - 24s 1s/step - loss: 0.4982 - accuracy: 0.8382 - val_loss: 0.5328 - val_accuracy: 0.8525
Epoch 3/30
18/18 [==============================] - 24s 1s/step - loss: 0.3833 - accuracy: 0.8745 - val_loss: 0.4763 - val_accuracy: 0.8689
Epoch 4/30
18/18 [==============================] - 23s 1s/step - loss: 0.3450 - accuracy: 0.8855 - val_loss: 0.4081 - val_accuracy: 0.8852
Epoch 5/30
18/18 [==============================] - 22s 1s/step - loss: 0.2452 - accuracy: 0.9073 - val_loss: 0.4904 - val_accuracy: 0.8689
Epoch 6/30
18/18 [==============================] - 23s 1s/step - loss: 0.2381 - accuracy: 0.9182 - val_loss: 0.5132 - val_accuracy: 0.8689
Epoch 7/30
18/18 [==============================] - 22s 1s/step - loss: 0.2514 - accuracy: 0.9127

In [14]:
# menyimpan model dalam format saved model
export_dir = 'saved_model/'
tf.saved_model.save(model, export_dir)

# convert SavedModel menjadi vegs.tflite
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

tflite_model_file = pathlib.Path('vegs.tflite')
tflite_model_file.write_bytes(tflite_model)

25245276